# Análisis de comportamiento de suscriptores de Megaline

La compañía móvil Megaline quiere desarrollar un modelo para analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra. Se busca un modelo con la mayor exactitud posible (umbral de exactitud de 0.75)

Se tiene acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos.

## Inicialización
### Cargar librerias

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 

### Cargar y explorar datos

In [2]:
#Cargar archivos de datos
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#Explorar datos
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 3214 entries, 0 to 3213

Data columns (total 5 columns):

 #   Column    Non-Null Count  Dtype  

---  ------    --------------  -----  

 0   calls     3214 non-null   float64

 1   minutes   3214 non-null   float64

 2   messages  3214 non-null   float64

 3   mb_used   3214 non-null   float64

 4   is_ultra  3214 non-null   int64  

dtypes: float64(4), int64(1)

memory usage: 125.7 KB


Cada fila del dataset corresponde a la información del comportamiento mensual sobre un usuario. 

La información dada es la siguiente:
- `сalls` — número de llamadas,
- `minutes` — duración total de las llamadas en minutos,
- `messages` — número de mensajes de texto,
- `mb_used` — Tráfico de Internet utilizado en MB,
- `is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0).

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
#vista previa del df
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


El dataframe anterior, no tiene datos ausentes y los datos son del tipo adecuado a excepción de la columna `messages` que deberia ser int y es float.

La columna `is_ultra` tiene datos booleanos pero se maniente como int para facilitar el análisis.

### Corregir datos

In [6]:
#cambiar de float a int
df['messages'] = df.astype('int')

In [7]:
#verificar cambio
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 3214 entries, 0 to 3213

Data columns (total 5 columns):

 #   Column    Non-Null Count  Dtype  

---  ------    --------------  -----  

 0   calls     3214 non-null   float64

 1   minutes   3214 non-null   float64

 2   messages  3214 non-null   int64  

 3   mb_used   3214 non-null   float64

 4   is_ultra  3214 non-null   int64  

dtypes: float64(3), int64(2)

memory usage: 125.7 KB


No hay valores ausentes en esta tabla y los datos son del tipo correcto.

## Crear modelos de aprendizaje
A continuación se crean distintos modelos de aprendizaje automático para después compararlos.

Se tiene en cuenta que el objetivo es una respuesta de categoría.

### Segmentar datos
Se segmentan los datos fuente en 3 conjuntos:
- conjunto de entrenamiento (60%)
- conjunto de validación (20%)
- conjunto de prueba (20%)

In [8]:
#Separar conjunto de datos de prueba
df_t, df_test = train_test_split(df,test_size=0.2,random_state=12345)

In [9]:
#Separar conjunto de datos de entrenamiento y validacion. El test_size sube para compensar la reducción de filas
df_train, df_valid = train_test_split(df_t,test_size=0.25,random_state=12345)

In [10]:
#Separar variables de conjunto de entrenamiento
df_train_features = df_train.drop('is_ultra',axis=1)
df_train_target =  df_train['is_ultra']

In [11]:
#Separar variables de conjunto de validación
df_valid_features = df_valid.drop('is_ultra',axis=1)
df_valid_target = df_valid['is_ultra']

In [12]:
#Separar variables de conjunto de prueba
df_test_features = df_test.drop('is_ultra',axis=1)
df_test_target = df_test['is_ultra']

In [13]:
#verificar tamaño de los datasets
print(df_test.shape)
print(df_train.shape)
print(df_valid.shape)

(643, 5)

(1928, 5)

(643, 5)


### Modelo: Árbol de decisión para clasificación

In [14]:
#experimentar con diferentes opciones de profundidad de árbol para buscar la mejor
max_depth = 0
best_accuracy = 0

for depth in range(1, 11): 
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(df_train_features, df_train_target) 
    predictions_valid = model.predict(df_valid_features) 
    accuracy_s = accuracy_score(df_valid_target, predictions_valid)
    print('max_depth =', depth, ': ',accuracy_s) 
    
    if accuracy_s > best_accuracy:
        best_accuracy = accuracy_s
        max_depth = depth
print()
print('Best max_depth =', max_depth, ': ',best_accuracy)

max_depth = 1 :  0.7387247278382582

max_depth = 2 :  0.7573872472783826

max_depth = 3 :  0.7620528771384136

max_depth = 4 :  0.76049766718507

max_depth = 5 :  0.7527216174183515

max_depth = 6 :  0.7558320373250389

max_depth = 7 :  0.7636080870917574

max_depth = 8 :  0.7527216174183515

max_depth = 9 :  0.7480559875583204

max_depth = 10 :  0.749611197511664



Best max_depth = 7 :  0.7636080870917574


El modelo cambia ligeramente al modificar el parámetro `max_depth`.

Se establece `max_depth = 7`. El modelo cumple con umbral de exactitud > 0.75 .

In [15]:
#usar GridSearchCV para buscar la mejor profundidad de árbol
param_grid = {'max_depth': [1, 2, 3, 4, 5,6,7,8,9,10,20,30,40],'criterion':['gini','entropy'], }

tree_clas = DecisionTreeClassifier(random_state=12345)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(df_train_features, df_train_target)

final_model = grid_search.best_estimator_
final_model

Fitting 5 folds for each of 26 candidates, totalling 130 fits


DecisionTreeClassifier(max_depth=3, random_state=12345)

In [16]:
# exactitud del modelo con gridsearch
accuracy_score(df_valid_target, final_model.predict(df_valid_features) )

0.7620528771384136

Este método dio como resultado `max_depth = 3`, sin embargo `max_depth = 7` da una mejor exactitud.

In [17]:
#fijar modelo
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=7) 
model_dtc.fit(df_train_features, df_train_target) 

predictions_valid_dtc = model_dtc.predict(df_valid_features) 

#error
rmse_dtc = mean_squared_error(df_valid_target, predictions_valid_dtc)**0.5
rmse_dtc

0.48620151471199946

In [18]:
#Precicciones
train_predictions = model_dtc.predict(df_train_features) 
test_predictions = model_dtc.predict(df_test_features) 

#comprobar calidad del modelo
acc_tr_dtc  = accuracy_score(df_train_target, train_predictions)
acc_val_dtc = best_accuracy
acc_ts_dtc  = accuracy_score(df_test_target, test_predictions)

print('Exactitud') 
print('Training set:', acc_tr_dtc)
print('Validation set:', acc_val_dtc)
print('Test set:', acc_ts_dtc) 

Exactitud

Training set: 0.8340248962655602

Validation set: 0.7636080870917574

Test set: 0.7916018662519441


Se mantiene exactitud > 0.75 .

### Modelo: Bosque aleatorio para clasificación

In [19]:
#buscar las mejores características para los datos disponibles
model_rfc =RandomForestClassifier(random_state=12345)
param_grid = {'n_estimators': [1,2,3,4,5,6,7,8,9,10], 'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth' : [3,4,5,6,7,8,9,10]}
CV_rfc = GridSearchCV(estimator=model_rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(df_train_features,df_train_target)
CV_rfc.best_params_

{'max_depth': 4, 'max_features': 'auto', 'n_estimators': 4}

In [20]:
#fijar modelo con parámetros encontrados
model_rfc = RandomForestClassifier(random_state=12345, max_depth=4, max_features='auto',n_estimators=4)
model_rfc.fit(df_train_features,df_train_target)  

predictions_valid_rfc = model_rfc.predict(df_valid_features) 

#error
rmse_rfc = mean_squared_error(df_valid_target, predictions_valid_rfc)**0.5
rmse_rfc

0.4845995284303307

In [21]:
#predicciones
train_predictions = model_rfc.predict(df_train_features) 
test_predictions = model_rfc.predict(df_test_features)

#comprobar calidad del modelo
acc_tr_rfc = accuracy_score(df_train_target, train_predictions)
acc_val_rfc = accuracy_score(df_valid_target, predictions_valid_rfc)
acc_ts_rfc = accuracy_score(df_test_target, test_predictions)

print('Exactitud') 
print('Training set:', acc_tr_rfc)
print('Validation set:', acc_val_rfc)
print('Test set:', acc_ts_rfc) 

Exactitud

Training set: 0.8029045643153527

Validation set: 0.7651632970451011

Test set: 0.7807153965785381


Se tiene exactitud > 0.75.

### Modelo: Regresión logística

In [22]:
#inicializar modelo y fijarlo
model_lr = LogisticRegression(random_state=12345, solver='liblinear') 
model_lr.fit(df_train_features,df_train_target)
 
predictions_valid_lr = model_lr.predict(df_valid_features) 

#error
rmse_lr = mean_squared_error(df_valid_target, predictions_valid_lr)**0.5
rmse_lr

0.5563153608479606

In [23]:
#predicciones
train_predictions = model_lr.predict(df_train_features) 
test_predictions = model_lr.predict(df_test_features)

#comprobar calidad del modelo con el conjunto de prueba
acc_tr_lr = accuracy_score(df_train_target, train_predictions)
acc_val_lr = accuracy_score(df_valid_target, predictions_valid_lr)
acc_ts_lr = accuracy_score(df_test_target, test_predictions)

print('Accuracy') 
print('Training set:', acc_tr_lr)
print('Validation set:', acc_val_lr)
print('Test set:', acc_ts_lr) 

Accuracy

Training set: 0.6955394190871369

Validation set: 0.6905132192846034

Test set: 0.6967340590979783


El modelo muestra una exactitud menor a la exactitud mínima requerida que es 0.75.

## Prueba de cordura del modelo elegido

A continuación se aplica una prueba de cordura.

In [24]:
#olumna objetivo
target = df['is_ultra']

#valor promedio de la columna objetivo
predictions = pd.Series(target.mean(), index=target.index)

#error cuadrático medio ECM
mse = mean_squared_error(target, predictions) 

#raíz cuadrada de ECM
rmse = mean_squared_error(target, predictions,squared=False) 
print('RMSE:', rmse) 

RMSE: 0.46102797293043907


## Conclusión
En base a los datos estudiados, se crearon 3 modelos, los cuales tuvieron los siguientes resultados:

In [25]:
#tabla de resultados por modelo
pd.DataFrame({'Model':['Decision Tree Classifier', 'Random Forest Classifier', 'Logistic Regression'],
    'Training set accuracy':  [acc_tr_dtc    , acc_tr_rfc   , acc_tr_lr ],
    'Validation set accuracy':[acc_val_dtc   , acc_val_rfc  , acc_val_lr],
    'Test set accuracy':      [acc_ts_dtc    , acc_ts_rfc   , acc_ts_lr ],
    'RMSE Validation set':    [rmse_dtc      , rmse_rfc     ,  rmse_lr  ]  })

,Model,Training set accuracy,Validation set accuracy,Test set accuracy,RMSE Validation set
0,Decision Tree Classifier,0.834025,0.763608,0.791602,0.486202
1,Random Forest Classifier,0.802905,0.765163,0.780715,0.484600
2,Logistic Regression,0.695539,0.690513,0.696734,0.556315


- Los modelos árbol de decisión para clasificación y bosque aleatorio para clasificación muestran alta exactitud en los tres conjuntos de datos. 

- El modelo de regresión lógistica tiene una exactitud menor a la deseada (0.75) en todos los conjuntos de datos.

Por lo anterior mencionado y tomando en cuenta que el modelo **árbol de decisión para clasificación** es el de más alta velocidad de procesamiento, se escoge este como el mejor modelo para analizar el comportamiento de los clientes y hacerles recomendaciones adecuadas.